In [ ]:
!pip install cbsodata
import cbsodata
import pandas as pd
import torch
from transformers import pipeline, BartTokenizer
from sentence_transformers import SentenceTransformer, util

In [ ]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize the summarizer and sentence transformer
summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=device)
embedding_model = SentenceTransformer('paraphrase-MiniLM-L6-v2').to(device)
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

# Function to summarize a long description by chunking it
def summarize_long_description(description, max_length=1024, chunk_size=500):
    """Summarize long description by splitting into smaller chunks."""
    tokens = tokenizer.encode(description, truncation=False)
    if len(tokens) <= max_length:
        # If it's small enough, just summarize directly
        return summarizer(description, max_length=200, min_length=50, do_sample=False)[0]['summary_text']

    # Otherwise, split into chunks
    chunks = [tokens[i:i + chunk_size] for i in range(0, len(tokens), chunk_size)]
    summaries = []
    for chunk in chunks:
        chunk_text = tokenizer.decode(chunk, skip_special_tokens=True)
        summary = summarizer(chunk_text, max_length=200, min_length=50, do_sample=False)[0]['summary_text']
        summaries.append(summary)

    # Combine all chunk summaries into one
    return " ".join(summaries)

# Function to extract only the English table descriptions
def extract_english_descriptions(tables):
    descriptions = []
    for table in tables:
        # Process only English tables
        if table.get('Language', '').endswith('en'):
            # Main description of the table
            table_title = table.get('Title', 'Unknown Table Title').strip()
            table_description = table.get('ShortDescription', '').strip()
            descriptions.append({
                "table_title": table_title,
                "table_description": table_description,
                "Identifier": table.get('Identifier')
            })
    return descriptions

# Function to process and summarize table and column descriptions separately
def process_table_and_column_descriptions(descriptions):
    table_embeddings = []
    column_embeddings = []
    table_to_column_mapping = {}

    for desc in descriptions:
        #print(desc['Identifier'])
        table_id = desc['Identifier']
        table_title = desc['table_title']
        table_description = desc['table_description']
        try:
            # Summarize table description
            summarized_table_desc = summarize_long_description(table_description)
            combined_table_text = f"{table_title}: {summarized_table_desc}"  # Title + Summary

            # Retrieve column metadata
            metadata = pd.DataFrame(cbsodata.get_meta(table_id, name='DataProperties'))

            # Embed table description (title + description)
            table_embedding = embedding_model.encode(combined_table_text, convert_to_tensor=True)
            table_embeddings.append({"table_id": table_id, "embedding": table_embedding})

            # Process column titles and create embeddings
            column_titles = []
            for _, row in metadata.iterrows():
                try:
                    column_title = row.get('Title', '').strip()
                    # Skip if column title is empty or None
                    if not column_title:
                        continue
                    column_titles.append(column_title)
                except Exception as e:
                    print(f"Skipping column in table {table_id} due to error: {e}")

            # Embed column titles
            if column_titles:
                column_embedding = embedding_model.encode(column_titles, convert_to_tensor=True)
                column_embeddings.append({"table_id": table_id, "column_embeddings": column_embedding})
                table_to_column_mapping[table_id] = column_titles

        except Exception as e:
            print(f"Skipping table {table_id} due to error: {e}")

    return table_embeddings, column_embeddings, table_to_column_mapping

# Set display options for more text
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# DATA EXPLORATION
tables = cbsodata.get_table_list()

# Extract English table descriptions
descriptions = extract_english_descriptions(tables)

# Process and summarize table and column metadata
table_embeddings, column_embeddings, table_to_column_mapping = process_table_and_column_descriptions(descriptions)

# Example Output
print("Example Table Embeddings:")
for table in table_embeddings[:3]:  # Display first 3 tables for brevity
    print(f"Table ID: {table['table_id']}")
    print(f"Table Embedding: {table['embedding']}")

print("\nExample Column Embeddings:")
for column in column_embeddings[:3]:  # Display first 3 tables for brevity
    print(f"Table ID: {column['table_id']}")
    print(f"Column Embeddings: {column['column_embeddings']}")

print("\nExample Table to Column Mapping:")
for table_id, columns in list(table_to_column_mapping.items())[:3]:  # Display first 3 mappings
    print(f"Table ID: {table_id}")
    print(f"Columns: {columns}")


80783eng


Your max_length is set to 200, but your input_length is only 45. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=22)


80784eng


Your max_length is set to 200, but your input_length is only 39. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=19)


85636ENG


...

37112ENG


Your max_length is set to 200, but your input_length is only 173. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=86)


81821ENG
80079eng


Your max_length is set to 200, but your input_length is only 149. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=74)


81157eng
71844eng
Example Table Embeddings:
Table ID: 80783eng
Table Embedding: tensor([ 3.3498e-02, -2.1545e-01, -3.7267e-02, -1.3228e-01,  1.4134e-01,
        -4.4189e-02, -3.3582e-01, -2.7006e-01,  2.7065e-02,  2.0547e-01,
         1.3194e-01, -6.0072e-02, -8.6440e-02, -7.4068e-02, -1.1227e-01,
        -2.4221e-01,  3.8034e-02, -6.9142e-02,  1.2743e-01, -3.8545e-02,
         8.5777e-02,  1.0851e-01, -2.8092e-01, -2.5289e-01,  1.3022e-01,
        -6.7360e-02, -2.9629e-01, -4.0961e-02, -7.0552e-02, -3.1285e-01,
         1.3497e-01,  1.5457e-01,  2.6439e-01,  1.6314e-01, -4.5641e-02,
        -6.0478e-02, -2.8252e-02, -1.6465e-01,  1.8010e-01,  3.2562e-01,
         7.3208e-02, -3.7041e-01,  2.5351e-01, -1.7549e-01,  1.2327e-02,
         3.4843e-01, -2.1140e-01, -5.3973e-02, -3.8470e-02,  7.6663e-02,
        -3.0450e-01,  1.4997e-01, -5.8637e-02, -2.3188e-01,  6.8207e-02,
        -1.3918e-01,  5.3255e-02, -2.0161e-01, -2.3913e-01, -1.8370e-01,
         1.0635e-01,  2.1791e-01, -1.3700e-0

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# Save to Google Drive
torch.save(table_embeddings, '/content/drive/My Drive/table_embeddings.pt')
torch.save(column_embeddings, '/content/drive/My Drive/column_embeddings.pt')
torch.save(table_to_column_mapping, '/content/drive/My Drive/table_to_column_mapping.pt')
print("Embeddings and mappings saved to Google Drive!")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Embeddings and mappings saved to Google Drive!
